# Part 2

In [142]:
from sklearn.metrics import confusion_matrix
# For train data
file = open("reviewstrain.txt", "r") 
all_comments = file.readlines()
all_comments[0][:-2]
train_label = []
train_data = []
for comment in all_comments:
    label = comment[0]
    data = list(set(comment[2:-2].split()))
    train_data.append(data)
    train_label.append(label)

In [143]:
# For test data
file_test = open("reviewstest.txt", "r") 
all_comments = file_test.readlines()
test_label = []
test_data = []
for comment in all_comments:
    label = comment[0]
    data = list(set(comment[2:-2].split()))
    test_data.append(data)
    test_label.append(label)

In [157]:
def KNN(test_data, train_data, train_label, k):
    predicted = []
    for tes_data in test_data:
        dictionary = {}
        temp = []
        for index,tra_data in list(enumerate(train_data)):
            
            # Calculate Distance
            inter_c1_c2 = len(set(tes_data) - (set(tes_data)-set(tra_data)))
            if inter_c1_c2 == 0:
                distance = 1
            else:
                distance = 1/inter_c1_c2
            
            # Store dictionary with dist as key and labels as values
            if distance not in dictionary:
                dictionary[distance] = [train_label[index]]
            else:
                dictionary[distance].append(train_label[index])
                
        
        sorted_keys = sorted(dictionary)
        
        if k == 1:
            if len(dictionary[sorted_keys[0]]) == 1:
                predicted.append(int(dictionary[sorted_keys[0]][0]))
            else:
                count_1 = dictionary[sorted_keys[0]].count('1')
                count_0 = dictionary[sorted_keys[0]].count('0')
                if count_0>count_1:
                    predicted.append(0)
                else:
                    predicted.append(1)
                    
        else:
            temp = 0
            distances_to_traverse = []
            for i in range(len(dictionary)):
                temp+=len(dictionary[sorted_keys[i]])
                distances_to_traverse.append(sorted_keys[i])
                if temp>=k:
                    break
            l = []
            for dist in distances_to_traverse:
                l += dictionary[dist]    
    
            count_1 = l.count('1')
            count_0 = l.count('0')
            if count_0>count_1:
                predicted.append(0)
            else:
                predicted.append(1)
                    
    return predicted


answer = KNN(test_data,train_data,train_label,5)

#### Accuracy for k=5:

In [158]:
def accuracy(predicted, real):
    temp = 0
    for p,r in zip(predicted, real):
        if int(p)==int(r):
            temp+=1
            
    return (temp/len(predicted))*100

print(accuracy(answer,test_label))

60.6


#### Confusion Matrix for k=5:

In [159]:
test_label = list(map(int,test_label))
answer = list(map(int,answer))

confusion_matrix(test_label, answer)

array([[ 91, 136],
       [ 61, 212]])

In [160]:
answer[17]

1

# Cross Validation

In [161]:
length_of_each_data = len(train_label)//5

for k in k_values:
    
    predicted_cv_final = [0]*len(train_label)
    
    for i in range(1,6):
        
        start_index = length_of_each_data*(i-1)
        end_index = length_of_each_data*i
        
        
        
        validation_data_cv = train_data [start_index : end_index]
        validation_label_cv = train_label [start_index : end_index]

        if i==1:
            train_data_cv = train_data [end_index:]
            train_label_cv = train_label [end_index:]
        else:
            train_data_cv = train_data[:start_index] + train_data[end_index:]
            train_label_cv = train_label[:start_index] + train_label[end_index:]


        predicted_cv = KNN(validation_data_cv, train_data_cv, train_label_cv, k)
        
        predicted_cv_final[start_index:end_index] = predicted_cv
        
    print("For k = {}, accuracy = {}".format(k,accuracy(predicted_cv_final,train_label)))

For k = 3, accuracy = 66.0
For k = 7, accuracy = 65.8
For k = 99, accuracy = 61.133333333333326


#### Accuracy & Confusion Matrix for k=3:

In [162]:
answer = KNN(test_data,train_data,train_label,3)
print(accuracy(answer,test_label))

59.0


In [163]:
test_label = list(map(int,test_label))
answer = list(map(int,answer))

confusion_matrix(test_label, answer)

array([[ 83, 144],
       [ 61, 212]])

#### Our own distance function:

In [1]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [194]:
# For train data
file = open("reviewstrain.txt", "r") 
all_comments = file.readlines()
all_comments[0][:-2]
train_label = []
train_data = []
for comment in all_comments:
    label = comment[0]
    data = comment[2:-2]
    train_data.append(data)
    train_label.append(label)

In [195]:
# For test data
file_test = open("reviewstest.txt", "r") 
all_comments = file_test.readlines()
test_label = []
test_data = []
for comment in all_comments:
    label = comment[0]
    data = comment[2:-2]
    test_data.append(data)
    test_label.append(label)

In [186]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [191]:

import string
def KNN(test_data, train_data, train_label, k):
    predicted = []
    for tes_data in test_data:
        dictionary = {}
        temp = []
        for index,tra_data in list(enumerate(train_data)):
            
            temp_tra_data = tra_data.split()
            tra_data = [w for w in temp_tra_data if w not in string.punctuation]
            tra_data = "".join(tra_data)
            
            
            temp_tes_data = tes_data.split()
            tes_data = [w for w in temp_tes_data if w not in string.punctuation]
            tes_data = "".join(tes_data)
            
            distance = levenshteinDistance(tra_data,tes_data)
            distance = round(distance,1)
            
#             distance = distance_sequenceMatcher(str(tra_data),str(tes_data))
            
#             distance = round(distance,1)
            
            # Store dictionary with dist as key and labels as values
            if distance not in dictionary:
                dictionary[distance] = [train_label[index]]
            else:
                dictionary[distance].append(train_label[index])
                
        
        sorted_keys = sorted(dictionary)
        
        if k == 1:
            if len(dictionary[sorted_keys[0]]) == 1:
                predicted.append(int(dictionary[sorted_keys[0]][0]))
            else:
                count_1 = dictionary[sorted_keys[0]].count('1')
                count_0 = dictionary[sorted_keys[0]].count('0')
                if count_0>count_1:
                    predicted.append(0)
                else:
                    predicted.append(1)
                    
        else:
            temp = 0
            distances_to_traverse = []
            for i in range(len(dictionary)):
                temp+=len(dictionary[sorted_keys[i]])
                distances_to_traverse.append(sorted_keys[i])
                if temp>=k:
                    break
            l = []
            for dist in distances_to_traverse:
                l += dictionary[dist]    
    
            count_1 = l.count('1')
            count_0 = l.count('0')
            if count_0>count_1:
                predicted.append(0)
            else:
                predicted.append(1)
                    
    return predicted


answer = KNN(test_data,train_data,train_label,1)
print("Accuracy = ",accuracy(answer,test_label))

Accuracy =  61.6


In [193]:
test_label = list(map(int,test_label))
answer = list(map(int,answer))

confusion_matrix(test_label, answer)

array([[ 91, 136],
       [ 56, 217]])

In [196]:
from nltk.corpus import stopwords
nltk_words = list(stopwords.words('english'))
import string
def KNN(test_data, train_data, train_label, k):
    predicted = []
    for tes_data in test_data:
        dictionary = {}
        temp = []
        for index,tra_data in list(enumerate(train_data)):
            
            temp_tra_data = tra_data.split()
            tra_data = [w for w in temp_tra_data if w not in string.punctuation]
            tra_data = "".join(tra_data)
            
            
            temp_tes_data = tes_data.split()
            tes_data = [w for w in temp_tes_data if w not in string.punctuation]
            tes_data = "".join(tes_data)
            
            distance = levenshteinDistance(tra_data,tes_data)
            distance = round(distance,1)
            
#             distance = distance_sequenceMatcher(str(tra_data),str(tes_data))
            
#             distance = round(distance,1)
            
            # Store dictionary with dist as key and labels as values
            if distance not in dictionary:
                dictionary[distance] = [train_label[index]]
            else:
                dictionary[distance].append(train_label[index])
                
        
        sorted_keys = sorted(dictionary)
        
        if k == 1:
            if len(dictionary[sorted_keys[0]]) == 1:
                predicted.append(int(dictionary[sorted_keys[0]][0]))
            else:
                count_1 = dictionary[sorted_keys[0]].count('1')
                count_0 = dictionary[sorted_keys[0]].count('0')
                if count_0>count_1:
                    predicted.append(0)
                else:
                    predicted.append(1)
                    
        else:
            temp = 0
            distances_to_traverse = []
            for i in range(len(dictionary)):
                temp+=len(dictionary[sorted_keys[i]])
                distances_to_traverse.append(sorted_keys[i])
                if temp>=k:
                    break
            l = []
            for dist in distances_to_traverse:
                l += dictionary[dist]    
    
            count_1 = l.count('1')
            count_0 = l.count('0')
            if count_0>count_1:
                predicted.append(0)
            else:
                predicted.append(1)
                    
    return predicted


answer = KNN(test_data,train_data,train_label,5)
print("Accuracy = ",accuracy(answer,test_label))

Accuracy =  64.0


In [197]:
test_label = list(map(int,test_label))
answer = list(map(int,answer))

confusion_matrix(test_label, answer)

array([[ 84, 143],
       [ 37, 236]])